In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

# use CMU Serif
import matplotlib as mpl
import matplotlib.font_manager as font_manager
mpl.rcParams['font.family']='serif'
cmfont = font_manager.FontProperties(fname=mpl.get_data_path() + '/fonts/ttf/cmr10.ttf')
# cmfont = font_manager.FontProperties('/Users/theoares/Library/Fonts/cmunrm.otf')
mpl.rcParams['font.serif']=cmfont.get_name()
mpl.rcParams['mathtext.fontset']='cm'
mpl.rcParams['axes.unicode_minus']=False

from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from matplotlib.transforms import Bbox

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
import h5py
import os
import itertools

n_boot = 200
from utils import *

## Confirm npr_momfrac output is the same as what we have here

In [7]:
mu = 0
f_npr['O' + str(mu + 1) + str(mu + 1)][()]

TypeError: Accessing a group is done with bytes or str,  not <class 'tuple'>

In [10]:
f_npr['O11'].keys()

<KeysViewHDF5 ['p1111', 'p3000']>

In [27]:
k_list = np.array([[1, 1, 1, 1], [3, 0, 0, 0]])
k_list_str = [klist_to_string(k, 'p') for k in k_list]
L = Lattice(12, 24)
npr_path = '/Users/theoares/Dropbox (MIT)/research/gq_mixing/meas/tests/npr_momfrac_test/cfg1010_npr_momfrac.h5'
gq_path = '/Users/theoares/Dropbox (MIT)/research/gq_mixing/meas/tests/npr_momfrac_test/cfg1010_gq_mixing.h5'
Gmu_npr = np.zeros((4, 4, 2, 4, 4, 3, 3), dtype = np.complex64)
Ga_gq = np.zeros((3, 2, 4, 4, 3, 3), dtype = np.complex64)
f_npr = h5py.File(npr_path, 'r')
for mu in range(4):
    for ii, kstr in enumerate(k_list_str):
        Gmu_npr[mu, mu, ii] = f_npr['O' + str(mu + 1) + str(mu + 1) + '/' + kstr][()]
f_npr.close()
f_gq = h5py.File(gq_path, 'r')
for a in range(3):
    for ii, kstr in enumerate(k_list_str):
        Ga_gq[a, ii] = f_gq['Gqq/3' + str(a + 1) + '/' + kstr][()]
f_gq.close()

In [37]:
Ga_npr = form_2d_sym_irreps(Gmu_npr)[0]
print(np.max(np.abs(Ga_npr - Ga_gq)))

8.4293696e-08


The results from running npr_momfrac.qlua and gq_mixing.qlua are consistent within inversion precision.

## Play with the Gqq, Gqg correlators

In [ ]:
l = 48
t = 96
L = Lattice(l, t)
V = (l ** 3) * t

In [ ]:
# read in qlua output
jobid = 90712
data_dir = '/Users/theoares/Dropbox (MIT)/research/gq_mixing/meas/cl21_48_96_b6p3_m0p2416_m0p2050_' + str(jobid)
unsorted_cfgs = []
unsorted_cfgids = []
for (dirpath, dirnames, file) in os.walk(data_dir):
    unsorted_cfgs.extend(file)
    for fi in file:
        unsorted_cfgids.append(int(fi[3 : 7]))    # slicing only good for cfgs in the 1000's
for idx, cfg in enumerate(unsorted_cfgs):
    unsorted_cfgs[idx] = data_dir + '/' + unsorted_cfgs[idx]
n_cfgs = len(unsorted_cfgs)
print('Reading ' + str(n_cfgs) + ' configs.')
print(unsorted_cfgids)

In [ ]:
# get glue pieces
glue_dir = '/Users/theoares/Dropbox (MIT)/research/gq_mixing/glue_ops/cfgs_1100_1500/'
path_glue_ids = glue_dir + 'ops_for_patrick.txt'
path_glue_emt = glue_dir + 'ops_for_patrick.npy'
glue_emt_all = np.load(path_glue_emt)
glue_ids = []
with open(path_glue_ids, 'r') as f:
    for line in f:
        glue_ids.append(int(line.split('cfg_')[1][:-1]))

In [ ]:
# sort all cfgs
cfgids = sorted(unsorted_cfgids)
cfgs = [x for _, x in sorted(zip(unsorted_cfgids, unsorted_cfgs), key = lambda pair: pair[0])]
glue_emt = np.array([glue_emt_all[ii] for ii in range(len(glue_ids)) if glue_ids[ii] in cfgids])

In [ ]:
# read in quark / glue stuff and bootstrap
k = np.array([3, 0, 0, 0])
props, Gqg_qlua, Gqq3, Gqq6 = readfiles(cfgs, k)
props_b = bootstrap(props)
glue_emt_b = bootstrap(glue_emt)
Gqq3_b = np.array([bootstrap(Gqq3[a]) for a in range(3)])
Gqq6_b = np.array([bootstrap(Gqq6[a]) for a in range(6)])
Gqg = np.einsum('zmn,zaibj->mnzaibj', glue_emt, props)
Gqg3, Gqg6 = form_2d_sym_irreps(Gqg)
Gqg3_b = np.array([bootstrap(Gqg3[a]) for a in range(3)])
Gqg6_b = np.array([bootstrap(Gqg6[a]) for a in range(6)])

In [ ]:
Gqq3_mu = np.mean(Gqq3_b, axis = 1)
Gqq3_sigma = np.std(Gqq3_b, axis = 1, ddof = 1)
Gqq6_mu = np.mean(Gqq6_b, axis = 1)
Gqq6_sigma = np.std(Gqq6_b, axis = 1, ddof = 1)
Gqg3_mu = np.mean(Gqg3_b, axis = 1)
Gqg3_sigma = np.std(Gqg3_b, axis = 1, ddof = 1)
Gqg6_mu = np.mean(Gqg6_b, axis = 1)
Gqg6_sigma = np.std(Gqg6_b, axis = 1, ddof = 1)

## See what the stats look like

In [ ]:
# try running on 4 nodes instead of 5-- not sure why the second job (89346) has bad data

In [ ]:
l = 48
t = 96
# L = Lattice(l, t)

In [ ]:
# path = '/Users/theoares/Dropbox (MIT)/research/gq_mixing/analysis_output/Z_89127.h5'
path = '/Users/theoares/Dropbox (MIT)/research/gq_mixing/analysis_output/Z_90712.h5'
f = h5py.File(path, 'r')
k_list = f['momenta'][()]
mom_list = [L.to_linear_momentum(k + bvec, datatype = np.float64) for k in k_list]
apsq_list = [square(p) for p in mom_list]
Zq = f['Zq'][()]
glue_emt = f['glue_emt'][()]
props = f['props'][()]
Gamma_qg3 = f['Gamma_qg3'][()]
Gamma_qg6 = f['Gamma_qg6'][()]
Gamma_qq3 = f['Gamma_qq3'][()]
Gamma_qq6 = f['Gamma_qq6'][()]
f.close()

In [ ]:
colors = ['b', 'b', 'b', 'b', 'r', 'r', 'r']

In [ ]:
Zq_mu = np.mean(Zq, axis = 1)
Zq_std = np.std(Zq, axis = 1, ddof = 1)
plt.figure()
plt.errorbar(apsq_list[:4], Zq_mu[:4], Zq_std[:4], c = 'r', fmt = '.', label = 'h(p) = 0.25')
plt.errorbar(apsq_list[4:], Zq_mu[4:], Zq_std[4:], c = 'b', fmt = '.', label = 'h(p) = 1.0')

plt.xlabel('$(ap)^2$')
plt.ylabel('$Z_q$')
plt.title('Quark renormalization on 32 configurations')
plt.legend()
plt.show()

In [ ]:
propsTr = np.einsum('pzaiai->pz', props)
propsTr_mu = np.mean(propsTr, axis = 1)
propsTr_std = np.std(propsTr, axis = 1, ddof = 1)

plt.figure()
plt.errorbar(apsq_list[:4], propsTr_mu[:4], propsTr_std[:4], c = 'r', fmt = '.', label = 'h(p) = 0.25')
plt.errorbar(apsq_list[4:], propsTr_mu[4:], propsTr_std[4:], c = 'b', fmt = '.', label = 'h(p) = 1.0')
plt.xlabel('$(ap)^2$')
plt.ylabel('$S$')
plt.title('Tr_D[Tr_C[S]] on 32 configurations.')
plt.show()

In [ ]:
props.shape

In [ ]:
props_inv = np.array([invert_props(props[p]) for p in range(len(apsq_list))])
propsInvTr = np.einsum('pzaiai->pz', props_inv)
propsInvTr_mu = np.mean(propsInvTr, axis = 1)
propsInvTr_std = np.std(propsInvTr, axis = 1, ddof = 1)

# TODO look at off diagonal, if S(p) ~ 0 then it might blow up

plt.figure()
plt.errorbar(apsq_list[:4], propsInvTr_mu[:4], propsInvTr_std[:4], c = 'r', fmt = '.', label = 'h(p) = 0.25')
plt.errorbar(apsq_list[4:], propsInvTr_mu[4:], propsInvTr_std[4:], c = 'b', fmt = '.', label = 'h(p) = 1.0')
plt.xlabel('$(ap)^2$')
plt.ylabel('$S^{-1}$')
plt.title('Tr_D[Tr_C[S^{-1}]] on 32 configurations.')
plt.show()

In [ ]:
glue_emt_mu = np.mean(glue_emt, axis = 0)
glue_emt_std = np.std(glue_emt, axis = 0, ddof = 1)
print('glue emt = ' + str(glue_emt_mu) + ' \pm ' + str(glue_emt_std))

In [ ]:
Gamma_qq3Tr = np.einsum('pyzaiai->ypz', Gamma_qq3)
Gamma_qq3Tr_mu = np.mean(Gamma_qq3Tr, axis = 2)
Gamma_qq3Tr_std = np.std(Gamma_qq3Tr, axis = 2, ddof = 1)

for a in range(3):
    plt.figure()
    plt.errorbar(apsq_list[:4], Gamma_qq3Tr_mu[a, :4], Gamma_qq3Tr_std[a, :4], c = 'r', fmt = '.', label = 'h(p) = 0.25')
    plt.errorbar(apsq_list[4:], Gamma_qq3Tr_mu[a, 4:], Gamma_qq3Tr_std[a, 4:], c = 'b', fmt = '.', label = 'h(p) = 1.0')

    plt.xlabel('$(ap)^2$')
    plt.ylabel('$ΓQQ$')
    plt.title('Tr_D[Tr_C[ΓQQ]] on 32 configurations, tau_1^3 irrep index ' + str(a))
    plt.legend()
    plt.show()

In [ ]:
glue_emt_b.shape

In [ ]:
glue_emt_mu.shape

In [ ]:
props_inv_mu.shape

In [ ]:
props_inv_mu = np.mean(props_inv, axis = 1)
normalization_mn = np.einsum('paibj,mn->mnpaibj', props_inv_mu, glue_emt_mu)
norm3, norm6 = form_2d_sym_irreps(normalization_mn)

In [ ]:
GamQG = np.einsum('pzaibj,zmn->mnpzaibj', props_inv, glue_emt_b)
GamQG_3, GamQG_6 = form_2d_sym_irreps(GamQG)
n_boot = props_inv.shape[1]
for b in range(n_boot):
    GamQG_3[:, :, b, :, :, :, :] = GamQG_3[:, :, b, :, :, :, :] - norm3[:, :, :, :, :, :]
    GamQG_6[:, :, b, :, :, :, :] = GamQG_6[:, :, b, :, :, :, :] - norm6[:, :, :, :, :, :]
# GamQG_3 = np.einsum('ypzaibj,ypaibj->ypzaibj', GamQG_3, 1 / norm3)
# GamQG_6 = np.einsum('ypzaibj,ypaibj->ypzaibj', GamQG_6, 1 / norm6)
GamQG_3Tr = np.einsum('ypzaiai->ypz', GamQG_3)
GamQG_3Tr_mu = np.mean(GamQG_3Tr, axis = 2)
GamQG_3Tr_std = np.std(GamQG_3Tr, axis = 2, ddof = 1)
# GamQG_3_mu = np.mean(GamQG_3, axis = 2)
# GamQG_3_std = np.std(GamQG_3, axis = 2, ddof = 1)

In [ ]:
plt.figure()
plt.errorbar(apsq_list[:4], GamQG_3Tr_mu[0, :4], GamQG_3Tr_std[0, :4], c = 'r', fmt = '.', label = 'h(p) = 0.25')
plt.errorbar(apsq_list[4:], GamQG_3Tr_mu[0, 4:], GamQG_3Tr_std[0, 4:], c = 'b', fmt = '.', label = 'h(p) = 1.0')
plt.title('Tr[S^{-1} O_g] - <S^{-1}> <O_g>')
plt.show()

In [ ]:
Gamma_qg3Tr = np.einsum('pyzaiai->ypz', Gamma_qg3)
Gamma_qg3Tr_mu = np.mean(Gamma_qg3Tr, axis = 2)
Gamma_qg3Tr_std = np.std(Gamma_qg3Tr, axis = 2, ddof = 1)

for a in range(3):
    plt.figure()
    plt.errorbar(apsq_list[:4], Gamma_qg3Tr_mu[a, :4], Gamma_qg3Tr_std[a, :4], c = 'r', fmt = '.', label = 'h(p) = 0.25')
    plt.errorbar(apsq_list[4:], Gamma_qg3Tr_mu[a, 4:], Gamma_qg3Tr_std[a, 4:], c = 'b', fmt = '.', label = 'h(p) = 1.0')

    plt.xlabel('$(ap)^2$')
    plt.ylabel('$ΓQG$')
    plt.title('Tr_D[Tr_C[ΓQG]] on 32 configurations, tau_1^3 irrep index ' + str(a))
    plt.legend()
    plt.show()

In [ ]:
# Gamma_qq3_red = np.abs(np.einsum('...aiaj->...ij', Gamma_qq3))
# Gamma_qq_red_mu = np.mean(Gamma_qq3_red, axis = 2)
# Gamma_qq_red_std = np.std(Gamma_qq3_red, axis = 2, ddof = 1)

In [ ]:
# plt.figure()
# plt.errorbar(apsq_list, Gamma_qq_red_mu[:, 0, 0, 0], Gamma_qq_red_std[:, 0, 0, 0], c = 'b', fmt = '.', label = '11')
# plt.errorbar(apsq_list, Gamma_qq_red_mu[:, 0, 0, 1], Gamma_qq_red_std[:, 0, 0, 1], c = 'r', fmt = '.', label = '12')
# plt.errorbar(apsq_list, Gamma_qq_red_mu[:, 0, 0, 2], Gamma_qq_red_std[:, 0, 0, 2], c = 'g', fmt = '.', label = '13')
# plt.errorbar(apsq_list, Gamma_qq_red_mu[:, 0, 0, 3], Gamma_qq_red_std[:, 0, 0, 3], c = 'k', fmt = '.', label = '14')

# plt.xlabel('$(ap)^2$')
# plt.ylabel('$Γ1_{1i}$')
# plt.title('Components of vertex function for $\mathcal{O}_{33} - \mathcal{O}_{44}$ on 15 configurations')
# plt.legend()
# plt.show()

## Playing with some actual data

In [ ]:
# cfgs which had gf problems: 1020, 1100, 1190
# cfgs which had issues writing measurement to file: 1110, 1140, 1180

In [ ]:
# for cfg_idx in range(1120, 1500, 10):
#     f2path = '/Users/theoares/Dropbox (MIT)/research/gq_mixing/meas/cl21_48_96_b6p3_m0p2416_m0p2050_90712/cfg' + str(cfg_idx) + '.h5'
#     try:
#         f2 = h5py.File(f2path, 'r')
#         print('good file at ' + str(cfg_idx))
#         print(np.einsum('iiaa->', f2['Gqq/31/p3000'][()]))
#         f2.close()
#     except:
#         print('broken file at ' + str(cfg_idx))

In [ ]:
# for cfg_idx in range(1120, 1500, 10):
#     f2path = '/Users/theoares/Dropbox (MIT)/research/gq_mixing/meas/cl21_48_96_b6p3_m0p2416_m0p2050_90712/cfg' + str(cfg_idx) + '.h5'
#     try:
#         f2 = h5py.File(f2path, 'r')
#         print('good file at ' + str(cfg_idx))
#         print(f2['Gqq'])
#         f2.close()
#     except:
#         print('broken file at ' + str(cfg_idx))

In [ ]:
# # f2path = '/Users/theoares/Dropbox (MIT)/research/gq_mixing/meas/cl21_48_96_b6p3_m0p2416_m0p2050_89157/cfg1110.h5'
# # f2path = '/Users/theoares/Dropbox (MIT)/research/gq_mixing/meas/cl21_48_96_b6p3_m0p2416_m0p2050_90712/cfg1240.h5'
# f2path = '/Users/theoares/Dropbox (MIT)/research/gq_mixing/meas/cfg1140.h5'
# f2 = h5py.File(f2path, 'r')
# print(f2['Gqq/31/p6000'][()])
# f2.close()

In [ ]:
# # some of the files are corrupted, which you can check in the slurm output (although they still wrote to the file). 
# # however, some of them are working fine. Going to try to run on the bad files with more nodes and see what happens
# # for the corrupted files, see what the correlator orders of magnitude are-- currently one Gqq for 1140 (one of the 
# # ones which errored) is having some major issues
# fpath = '/Users/theoares/Dropbox (MIT)/research/gq_mixing/meas/cl21_48_96_b6p3_m0p2416_m0p2050_89127/cfg1050.h5'
# f = h5py.File(fpath, 'r')
# print(f['Gqq/31/p6000'][()])

In [ ]:
# f['prop/p2222'][()]